In [42]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [43]:
train=pd.read_csv('train_emoji.csv',header=None)
train=train.drop(labels=[2,3],axis=1)

In [44]:
train.head()

,0,1
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [45]:
test=pd.read_csv('test_emoji.csv',header=None)

In [46]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [47]:
!pip install emoji


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import emoji

In [49]:
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [50]:
for i in emoji_dict.keys():
    print(i,end=" ")
    print(emoji.emojize(emoji_dict[i]))

0 :heart:
1 ⚾
2 :smile:
3 :disappointed:
4 🍴


In [51]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print ("-------------------------")
print (X_train[0],Y_train[0])

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [52]:
for i in range(X_train.shape[0]):
    X_train[i] = X_train[i].split(' ')
for i in range(X_test.shape[0]):
    X_test[i] = X_test[i].split()
Y_train = to_categorical(Y_train)


C:\Users\HP\AppData\Local\Temp\ipykernel_7468\128230943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[i] = X_train[i].split(' ')
C:\Users\HP\AppData\Local\Temp\ipykernel_7468\128230943.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[i] = X_test[i].split()


In [53]:
print (X_train[0],Y_train[0])


['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [54]:
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 24, 36, 19, 21, 13,  3,  3,  4], dtype=int64))

In [55]:
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]),
 array([ 3, 12, 16, 17,  3,  4,  1], dtype=int64))

In [58]:


embeddings_index = {}

# Open the file with utf-8 encoding
with open('glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Loaded {len(embeddings_index)} word vectors.')


Loaded 400000 word vectors.


In [59]:
embeddings_index["i"].shape

(50,)

In [60]:
from scipy import spatial
# Checking cosine similarity of words happy and sad
spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

0.31093674898147583

In [61]:
spatial.distance.cosine(embeddings_index["india"], embeddings_index["delhi"])

0.18572336435317993

In [62]:
spatial.distance.cosine(embeddings_index["france"], embeddings_index["paris"])

0.19746702909469604

In [65]:


# Assuming X_train and X_test are already defined
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        word = X_train[ix][ij].lower()
        if word in embeddings_index:
            embedding_matrix_train[ix][ij] = embeddings_index[word]
        else:
            embedding_matrix_train[ix][ij] = np.zeros(50)  # or any other default vector

for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        word = X_test[ix][ij].lower()
        if word in embeddings_index:
            embedding_matrix_test[ix][ij] = embeddings_index[word]
        else:
            embedding_matrix_test[ix][ij] = np.zeros(50)  # or any other default vector

print("Embedding matrices created successfully.")


Embedding matrices created successfully.


In [66]:
print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


# Training model using rnn

In [67]:
model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 10, 64)              │           7,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 10, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │             325 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 5)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,941 (62.27 KB)

 Trainable params: 15,941 (62.27 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.1542 - loss: 1.9970
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2530 - loss: 1.8440 
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3027 - loss: 1.6542 
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2740 - loss: 1.5858 
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3698 - loss: 1.4853 
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3870 - loss: 1.5105 
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3817 - loss: 1.4138 
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4165 - loss: 1.4297 
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4946 - loss: 1.4096 
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3737 - loss: 1.4046 
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5243 - loss: 1.2343 
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5022 - loss: 1.2210 
Ep

In [71]:
# Get the predicted probabilities for each class
pred_prob = model.predict(embedding_matrix_test)

# Convert probabilities to class labels
pred = np.argmax(pred_prob, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 349ms/step


In [72]:
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.5714285714285714

In [75]:
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]]),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]]))

2
['he', 'got', 'a', 'raise'] :disappointed: :smile:
3
['she', 'got', 'me', 'a', 'present'] :disappointed: :heart:
5
['he', 'is', 'a', 'good', 'friend'] :smile: :heart:
6
['I', 'am', 'upset'] :disappointed: :heart:
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 🍴 :heart:
12
['This', 'girl', 'is', 'messing', 'with', 'me'] :smile: :disappointed:
13
['are', 'you', 'serious', 'ha', 'ha'] :disappointed: :smile:
17
['Congratulation', 'for', 'having', 'a', 'baby'] :disappointed: :smile:
18
['stop', 'messing', 'around'] ⚾ :disappointed:
20
['I', 'love', 'taking', 'breaks'] 🍴 :heart:
21
['you', 'brighten', 'my', 'day'] 🍴 :smile:
23
['she', 'is', 'a', 'bully'] :smile: :disappointed:
26
['I', 'worked', 'during', 'my', 'birthday'] ⚾ :disappointed:
28
['enjoy', 'your', 'break'] 🍴 :smile:
29
['valentine', 'day', 'is', 'near'] ⚾ :heart:
30
['I', 'miss', 'you', 'so', 'much'] :disappointed: :heart:
35
['he', 'can', 'pitch', 'really', 'well'] :smile: ⚾
38
['See', 'you', 'at', 'the', 'restau

In [76]:
#for random sentence

In [77]:
x = ['i', 'do', 'think','this', 'class', 'is', 'very', 'interesting']

x_ = np.zeros((1,10,50))

for ix in range(len(x)):
    x_[0][ix] = embeddings_index[x[ix].lower()]

In [79]:
# Get the predicted probabilities for each class
pred_prob = model.predict(x_)

# Convert probabilities to class labels
pred = np.argmax(pred_prob, axis=1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


# Using LSTM

In [80]:
# A simple LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 10, 128)             │          91,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             645 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 5)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,877 (874.52 KB)

 Trainable params: 223,877 (874.52 KB)

 Non-trainable params: 0 (0.00 B)

In [81]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
# Training model
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.2404 - loss: 1.5826
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2690 - loss: 1.5126
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3396 - loss: 1.4540
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4656 - loss: 1.4175
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5488 - loss: 1.3048
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5589 - loss: 1.1831
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6799 - loss: 1.0676
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6767 - loss: 0.8938
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6901 - loss: 0.8022
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7322 - loss: 0.6793
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7702 - loss: 0.5642
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7165 - loss: 0.6379
E

In [84]:
# Get the predicted probabilities for each class
pred_prob = model.predict(embedding_matrix_test)

# Convert probabilities to class labels
pred = np.argmax(pred_prob, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 368ms/step


In [85]:
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.625

In [86]:
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]]),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]]))

3
['she', 'got', 'me', 'a', 'present'] :smile: :heart:
5
['he', 'is', 'a', 'good', 'friend'] :smile: :heart:
6
['I', 'am', 'upset'] :disappointed: :heart:
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] :smile: :heart:
11
['work', 'is', 'hard'] :smile: :disappointed:
12
['This', 'girl', 'is', 'messing', 'with', 'me'] :heart: :disappointed:
13
['are', 'you', 'serious', 'ha', 'ha'] :disappointed: :smile:
16
['work', 'is', 'horrible'] :smile: :disappointed:
19
['any', 'suggestions', 'for', 'dinner'] :smile: 🍴
21
['you', 'brighten', 'my', 'day'] :heart: :smile:
23
['she', 'is', 'a', 'bully'] :heart: :disappointed:
26
['I', 'worked', 'during', 'my', 'birthday'] :smile: :disappointed:
28
['enjoy', 'your', 'break'] ⚾ :smile:
29
['valentine', 'day', 'is', 'near'] :smile: :heart:
32
['My', 'life', 'is', 'so', 'boring'] :heart: :disappointed:
34
['will', 'you', 'be', 'my', 'valentine'] :smile: :heart:
37
['I', 'am', 'starving'] :disappointed: 🍴
41
['I', 'like', 'your', 'jacket'] :hea